In [1]:
from requests import post, get
import base64
import urllib
import pandas as pd 
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
client_id = os.environ["CLIENT_ID_SPOTIFY"]  

client_secret = os.environ["CLIENT_SECRET_SPOTIFY"] 

url = "https://accounts.spotify.com/api/token"


In [3]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
    }

    data = {
            "grant_type": "client_credentials"
            }

    result = post(url, headers=headers, data=data)

    return result.json()["access_token"]

In [4]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [5]:
def search_for_anything(token, artist_name, type):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"q={artist_name}&type={type}&limit=1"

    query_url = url + "?" + query 
    
    result = get(query_url, headers=headers)
    
    return result.json()


In [6]:
def list_playlist(token, playlist_id, limit=100, offset=0):
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    
    filter = "items(added_at,track(id, name, artists(name), popularity, duration_ms))"
    filter = urllib.parse.quote_plus(filter)
    filter = "fields="+filter
    filter = filter + f"&limit={limit}&offset={offset}"

    url = url + '?' + filter
    headers = get_auth_header(token)
    
    result = get(url, headers=headers)

    return result.json()

In [7]:
def get_total_music_playlist(token, id_playlist):
    url = f'https://api.spotify.com/v1/playlists/{id_playlist}?fields=tracks.total'
    headers = get_auth_header(token)
    result = get(url,headers=headers)

    return int(result.json()['tracks']['total'])



In [8]:
token = get_token()

In [9]:
def insert_musics_in_df(df, songs):
    for song in songs['items']:
        new_row = { "id_track":     song['track']['id'], 
                    "name":         song['track']['name'],
                    "added_at":     song['added_at'],
                    "artists":      ' - '.join([thing['name'] for thing in song['track']['artists']]),
                    "duration_ms":  song['track']['duration_ms'],
                    "popularity":   song['track']['popularity'],
                }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    
    return df


In [10]:
df = pd.DataFrame(columns=["id_track", "name", "added_at", "artists", "duration_ms", "popularity" ])

playlist = search_for_anything(token, "Radio Birita", type="playlist")
id_playlist = playlist['playlists']['items'][0]['id']
print(id_playlist)


16FFBX4MVut2qVV4SnTpZr


In [12]:
df = pd.DataFrame(columns=["id_track", "name", "added_at", "artists", "duration_ms", "popularity" ])

total_musica_playlist = get_total_music_playlist(token, id_playlist)

offset = 0
limit = 50
while offset < total_musica_playlist:
    songs = list_playlist(token, id_playlist, limit=limit, offset=offset)
    df = insert_musics_in_df(df, songs)

    offset+=50


In [14]:
df['added_at'] =  pd.to_datetime(df['added_at'])

In [18]:
df = df.sort_values(by=["added_at", "popularity"],ascending=False).reset_index(drop=True)

In [19]:
df

,id_track,name,added_at,artists,duration_ms,popularity
0,5fedkyJoFEIi6g7mPoOKdb,Apaixonadin,2023-09-15 16:57:40+00:00,Fernando Ciskinho Jr - João Bosco & Gabriel,130500,37
1,3QSsyYKRb8h768WoqZjfLo,Amor Demais / Saudade Dela - Ao Vivo,2023-09-15 04:51:42+00:00,Diego & Arnaldo,209534,23
2,4JUH9qIVVBUPWFm7PHhvIj,Me Libera / Pra Te Esquecer,2023-09-15 04:47:09+00:00,Day e Lara,217332,35
3,7ALwfzf8cq1lNeeAWefnTS,Parece Mas Não É - Ao Vivo,2023-09-11 06:13:49+00:00,Ícaro e Gilmar,145954,39
4,2yIJ2CVZkTDftAsL9NPTw2,Milionário - Ao Vivo,2023-09-11 05:04:56+00:00,Guilherme & Benuto - Wesley Safadão,154698,71
...,...,...,...,...,...,...
160,4nGDCINqy4PwlPDJWdDFHC,Mudei (Ao Vivo),2019-10-13 19:42:39+00:00,Villa Baggage,238598,30
161,6MQ7uenz2WKPKVoWdeEvwE,Prêmio de Consolação,2019-09-07 12:19:43+00:00,Matheus Neves & Renato - Fred & Gustavo,138307,21
162,1AUsF1MvGLwYHDG8bqCLTh,Resolva a Vida (Ao Vivo),2019-07-25 12:30:53+00:00,Marco Antonio & Gabriel - Bruninho & Davi,149573,0
163,7FXkdDfzPYZaEBhOA4H4L0,60 Dias Apaixonado / Sorriso Mudo - Ao Vivo,2019-05-15 05:41:03+00:00,Chitãozinho & Xororó,149160,18
